In [1]:
import torch.nn as nn
from attention import *


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\theju\anaconda3\envs\mlp\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Users\theju\anaconda3\envs\mlp\Lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
    app.start()
  File "c:\Users\theju\anaconda3\envs\mlp\Lib\site-packages\ipykernel\kernelapp.py", line 736, in start
    self.io_loop.start()
  File "c:\Users\the

In [2]:
query_shape = (3, 512)  # Example query tensor
key_shape = (3, 512)    # Example key tensor 
value_shape = (3, 512)  # Example value tensor
head_count = 8
model_size = 512
ffn_hidden_dimension = 4096

In [6]:
class Encoder(nn.Module):
    def __init__(self,query_shape,key_shape,value_shape,head_count,model_size,ffn_hidden_dimension):
        super().__init__()
        self.query_shape = query_shape
        self.key_shape = key_shape
        self.value_shape = value_shape
        self.head_count = head_count
        self.model_size = model_size
        self.ffn_hidden_dimension = ffn_hidden_dimension
        self.multi_head = MultiHeadAttention(self.query_shape,
                                             self.key_shape,
                                             self.value_shape,
                                             self.head_count,
                                             self.model_size)
        self.normal_layer_1 = nn.LayerNorm(self.model_size)
        self.normal_layer_2 = nn.LayerNorm(self.model_size)
        self.ffn = FeedForwardNetwork(self.model_size,self.model_size,
                                         self.ffn_hidden_dimension)
    
    def forward(self,x):
        input = x.detach().clone()
        layer_norm_1_output = self.normal_layer_1(input+self.multi_head(input,input,input))
        encoder_output = self.normal_layer_2(self.ffn(layer_norm_1_output))
        return encoder_output


In [7]:
encoder = Encoder(
                  query_shape,
                  key_shape,
                  value_shape,
                  head_count,
                  model_size,
                  ffn_hidden_dimension
                  )

In [8]:
x = torch.randn(3, 512) 
encoder_output = encoder(x)

In [9]:
encoder_output

tensor([[-0.3512, -0.6752, -0.5179,  ..., -0.6752, -0.6752, -0.6752],
        [ 3.3445, -0.6968, -0.6968,  ..., -0.6968, -0.6968, -0.6968],
        [-0.2786, -0.7371, -0.5614,  ..., -0.7371, -0.7371, -0.7371]],
       grad_fn=<NativeLayerNormBackward0>)